# 1. Generate Inteferece Graph

# 2. Make Prediction from DL4RegAlloc

In [9]:
from numpy.random import seed
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
seed(1)
tf.random.set_seed(2)

### Build Model

In [10]:
seqsize = 100
seq_inputs = layers.Input(shape=(seqsize,100,), dtype='float32')
encoder = layers.Bidirectional(layers.LSTM(512, return_sequences=True, name="lstm_1"))(seq_inputs)
encoder = layers.Dropout(0.2)(encoder)
encoder = layers.Bidirectional(layers.LSTM(256, return_sequences=True, name="lstm_2"))(encoder)
encoder = layers.Dropout(0.2)(encoder)
decoder = layers.Bidirectional(layers.LSTM(128, return_sequences=True, name="lstm_3"))(encoder)
decoder = layers.Dropout(0.2)(decoder)
decoderoutputs = layers.TimeDistributed(layers.Dense(101,activation="softmax"))(decoder)
model = tf.keras.Model(inputs=seq_inputs, outputs=decoderoutputs)

### Load Checkpoint

In [11]:
model = tf.keras.models.load_model("DL4RegAlloc/saved_model/color3.h5")

In [ ]:
# TESTING
csv_rows = []
test_csvs =["DL4RegAlloc/baidu.csv", "DL4RegAlloc/gkarate.csv"]
for i,csv in enumerate(test_csvs):
    print('\n------PREDICTING ',csv,'-------')
    predicted = model.predict(np.asarray(x2_pred_list[i]), batch_size=None, verbose=0, steps=None, callbacks=None)        
    print('\nInvalid edges percentage before color correction ->')
    post_process (np.asarray(x2_pred_list[i]), predicted)
    if ("ntst100.csv" in csv):
        print('')
    else:
        print('\nColors list and Chromatic number predicted by the model ->')
        colors_list_list_before_correction = post_process_chromatic(np.asarray(x2_pred_list[i]), predicted)
        print('\nApply color correction ->')
        predicted = post_process_correction(np.asarray(x2_pred_list[i]), predicted, colors_list_list_before_correction)
        print('\nColors list and Chromatic number following color correction ->')
        colors_list_list_after_correction = post_process_chromatic(np.asarray(x2_pred_list[i]), predicted)
        print('\nInvalid edges percentage after color correction ->')
        post_process (np.asarray(x2_pred_list[i]), predicted)
        create_csv_rows(csv.rsplit('/',1)[1], colors_list_list_before_correction, colors_list_list_after_correction)

import csv
with open(llvm_result_csv, 'w') as csvfile:
    # creating a csv writer object
    csvwriter = csv.writer(csvfile)      
    # writing the data rows
    csvwriter.writerows(csv_rows)

# 3. RegAlloc in LLVM